In [ ]:
pip install ultralytics

In [ ]:
pip install -U albumentations

In [30]:
import os
os.environ['WANDB_DISABLED'] = 'true'
from ultralytics import YOLO
import cv2
import albumentations as A
from tqdm import tqdm
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import classification_report
from PIL import Image
from pathlib import Path
import torchvision.models as models
import random
from collections import Counter
import shutil
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
# # 500 Samples. Split is 7(1/2 White BG - 1/2 Environment BG)/2/1

# def split_dataset(root_dir, output_dir, train_count=350, val_count=100, test_count=50):
#     total_count = train_count + val_count + test_count
#     train_white_bg_count = train_count // 2
#     train_env_bg_count = train_count - train_white_bg_count

#     train_dir = os.path.join(output_dir, 'train')
#     val_dir = os.path.join(output_dir, 'val')
#     test_dir = os.path.join(output_dir, 'test')

#     os.makedirs(train_dir, exist_ok=True)
#     os.makedirs(val_dir, exist_ok=True)
#     os.makedirs(test_dir, exist_ok=True)

#     class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

#     for class_dir in class_dirs:
#         class_path = os.path.join(root_dir, class_dir)
        
#         # Process White BG images for the training set
#         white_bg_path = os.path.join(class_path, 'White BG')
#         white_bg_images = []
#         if os.path.exists(white_bg_path):
#             white_bg_images = [f for f in os.listdir(white_bg_path) if os.path.isfile(os.path.join(white_bg_path, f)) and 'Discard' not in os.path.join(white_bg_path, f)]
#             random.shuffle(white_bg_images)
#             white_bg_images = white_bg_images[:train_white_bg_count]

#         os.makedirs(os.path.join(train_dir, class_dir), exist_ok=True)
#         for img in white_bg_images:
#             shutil.copy(os.path.join(white_bg_path, img), os.path.join(train_dir, class_dir, f"white_bg_{img}"))

#         # Process Environmental images
#         env_path = os.path.join(class_path, 'All')
#         env_images = [f for f in os.listdir(env_path) if os.path.isfile(os.path.join(env_path, f))]
#         random.shuffle(env_images)

#         # Ensure we have enough images
#         if len(env_images) < (train_env_bg_count + val_count + test_count):
#             print(f"Warning: Not enough images for {class_dir}. Only {len(env_images)} available.")
#             continue

#         test_images = env_images[:test_count]
#         val_images = env_images[test_count:test_count + val_count]
#         train_images = env_images[test_count + val_count:test_count + val_count + train_env_bg_count]

#         os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
#         for img in test_images:
#             shutil.copy(os.path.join(env_path, img), os.path.join(test_dir, class_dir, f"env_{img}"))

#         os.makedirs(os.path.join(val_dir, class_dir), exist_ok=True)
#         for img in val_images:
#             shutil.copy(os.path.join(env_path, img), os.path.join(val_dir, class_dir, f"env_{img}"))

#         os.makedirs(os.path.join(train_dir, class_dir), exist_ok=True)
#         for img in train_images:
#             shutil.copy(os.path.join(env_path, img), os.path.join(train_dir, class_dir, f"env_{img}"))

#     print("Dataset split completed.")
#     return train_dir, val_dir, test_dir, class_dirs


In [20]:
# # 500 Samples. Split is 7(1/3 White BG - 2/3 Environment BG)/2(1/3 White BG - 2/3 Environment BG)/1

# def split_dataset(root_dir, output_dir, train_count=350, val_count=100, test_count=50):
#     total_count = train_count + val_count + test_count
#     train_white_bg_count = train_count // 3
#     train_env_bg_count = train_count - train_white_bg_count
#     val_white_bg_count = val_count // 3
#     val_env_bg_count = val_count - val_white_bg_count

#     train_dir = os.path.join(output_dir, 'train')
#     val_dir = os.path.join(output_dir, 'val')
#     test_dir = os.path.join(output_dir, 'test')

#     os.makedirs(train_dir, exist_ok=True)
#     os.makedirs(val_dir, exist_ok=True)
#     os.makedirs(test_dir, exist_ok=True)

#     class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

#     for class_dir in class_dirs:
#         class_path = os.path.join(root_dir, class_dir)
        
#         # Process White BG images for the training and validation sets
#         white_bg_path = os.path.join(class_path, 'White BG')
#         white_bg_images = []
#         if os.path.exists(white_bg_path):
#             white_bg_images = [f for f in os.listdir(white_bg_path) if os.path.isfile(os.path.join(white_bg_path, f)) and 'Discard' not in os.path.join(white_bg_path, f)]
#             random.shuffle(white_bg_images)
#             white_bg_train_images = white_bg_images[:train_white_bg_count]
#             white_bg_val_images = white_bg_images[train_white_bg_count:train_white_bg_count + val_white_bg_count]

#         os.makedirs(os.path.join(train_dir, class_dir), exist_ok=True)
#         for img in white_bg_train_images:
#             shutil.copy(os.path.join(white_bg_path, img), os.path.join(train_dir, class_dir, f"white_bg_{img}"))

#         os.makedirs(os.path.join(val_dir, class_dir), exist_ok=True)
#         for img in white_bg_val_images:
#             shutil.copy(os.path.join(white_bg_path, img), os.path.join(val_dir, class_dir, f"white_bg_{img}"))

#         # Process Environmental images
#         env_path = os.path.join(class_path, 'All')
#         env_images = [f for f in os.listdir(env_path) if os.path.isfile(os.path.join(env_path, f))]
#         random.shuffle(env_images)

#         # Ensure we have enough images
#         if len(env_images) < (train_env_bg_count + val_env_bg_count + test_count):
#             print(f"Warning: Not enough images for {class_dir}. Only {len(env_images)} available.")
#             continue

#         test_images = env_images[:test_count]
#         val_images = env_images[test_count:test_count + val_env_bg_count]
#         train_images = env_images[test_count + val_env_bg_count:test_count + val_env_bg_count + train_env_bg_count]

#         os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
#         for img in test_images:
#             shutil.copy(os.path.join(env_path, img), os.path.join(test_dir, class_dir, f"env_{img}"))

#         os.makedirs(os.path.join(val_dir, class_dir), exist_ok=True)
#         for img in val_images:
#             shutil.copy(os.path.join(env_path, img), os.path.join(val_dir, class_dir, f"env_{img}"))

#         os.makedirs(os.path.join(train_dir, class_dir), exist_ok=True)
#         for img in train_images:
#             shutil.copy(os.path.join(env_path, img), os.path.join(train_dir, class_dir, f"env_{img}"))

#     print("Dataset split completed.")
#     return train_dir, val_dir, test_dir, class_dirs


In [26]:
# 400 Samples. Environment BG only 

def split_dataset(root_dir, output_dir, train_count=280, val_count=80, test_count=40):
    total_count = train_count + val_count + test_count

    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    for class_dir in class_dirs:
        class_path = os.path.join(root_dir, class_dir)

        # Process Environmental images
        env_path = os.path.join(class_path, 'All')
        env_images = [f for f in os.listdir(env_path) if os.path.isfile(os.path.join(env_path, f))]
        random.shuffle(env_images)

        # Ensure we have enough images
        if len(env_images) < total_count:
            print(f"Warning: Not enough images for {class_dir}. Only {len(env_images)} available.")
            continue

        test_images = env_images[:test_count]
        val_images = env_images[test_count:test_count + val_count]
        train_images = env_images[test_count + val_count:test_count + val_count + train_count]

        os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
        for img in test_images:
            shutil.copy(os.path.join(env_path, img), os.path.join(test_dir, class_dir, f"env_{img}"))

        os.makedirs(os.path.join(val_dir, class_dir), exist_ok=True)
        for img in val_images:
            shutil.copy(os.path.join(env_path, img), os.path.join(val_dir, class_dir, f"env_{img}"))

        os.makedirs(os.path.join(train_dir, class_dir), exist_ok=True)
        for img in train_images:
            shutil.copy(os.path.join(env_path, img), os.path.join(train_dir, class_dir, f"env_{img}"))

    print("Dataset split completed.")
    return train_dir, val_dir, test_dir, class_dirs

In [27]:
root_dir = r"E:\Jupyter\Snapfolia-Leaf-Detection-and-Classification-main\Samples\DATASET_3.1_Beta"
output_dir = "SPLITTED_DATA_YOLO_9_Samples"

In [28]:
train_dir, val_dir, test_dir, class_dirs = split_dataset(root_dir, output_dir)

Dataset split completed.


In [29]:
train_counts = Counter()
val_counts = Counter()
test_counts = Counter()

for class_dir in class_dirs:
    train_counts[class_dir] = len(os.listdir(os.path.join(train_dir, class_dir)))
    val_counts[class_dir] = len(os.listdir(os.path.join(val_dir, class_dir)))
    test_counts[class_dir] = len(os.listdir(os.path.join(test_dir, class_dir)))

# Print the counts
print("\nDataset Labels:")
print({class_name: idx for idx, class_name in enumerate(class_dirs)})


print("\nTotal Images per Class:")
for idx, class_name in enumerate(class_dirs):
    total = train_counts[class_name] + val_counts[class_name] + test_counts[class_name]
    print(f"{idx}    {class_name}: {total}")

print("\nImages per Class in TRAIN:")
for idx, (class_name, count) in enumerate(train_counts.items()):
    print(f"{idx}    {class_name}: {count}")

print("\nImages per Class in VALIDATION:")
for idx, (class_name, count) in enumerate(val_counts.items()):
    print(f"{idx}    {class_name}: {count}")

print("\nImages per Class in TEST:")
for idx, (class_name, count) in enumerate(test_counts.items()):
    print(f"{idx}    {class_name}: {count}")
    


Dataset Labels:
{'Apitong': 0, 'Balete': 1}

Total Images per Class:
0    Apitong: 400
1    Balete: 400

Images per Class in TRAIN:
0    Apitong: 280
1    Balete: 280

Images per Class in VALIDATION:
0    Apitong: 80
1    Balete: 80

Images per Class in TEST:
0    Apitong: 40
1    Balete: 40


In [ ]:
model = YOLO('yolov8n-cls.pt') 

In [ ]:
model.train(data='/Snapfolia - CS/YOLO v8/datasets/SPLITTED_DATA_YOLO_9_Samples', epochs=10, imgsz=640)

In [ ]:
results_path = './runs/classify/train33/results.csv'

results = pd.read_csv(results_path)

plt.figure()
plt.plot(results['                  epoch'], results['             train/loss'], label='train loss')
plt.plot(results['                  epoch'], results['               val/loss'], label='val loss', c='red')
plt.grid()
plt.title('Loss vs epochs')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()


plt.figure()
plt.plot(results['                  epoch'], results['  metrics/accuracy_top1'] * 100)
plt.grid()
plt.title('Validation accuracy vs epochs')
plt.ylabel('accuracy (%)')
plt.xlabel('epochs')

plt.show()

In [ ]:
from sklearn.metrics import classification_report
import os
from ultralytics import YOLO
import numpy as np
from PIL import Image
from torchvision import transforms

def generate_classification_report(run_dir, data_dir, classes):
    # Load the model
    model = YOLO(f'{run_dir}/weights/best.pt')
    
    # Prepare the test dataset
    test_dir = os.path.join(data_dir, 'test')
    test_images = []
    test_labels = []
    for class_id, class_name in enumerate(classes):
        class_dir = os.path.join(test_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            test_images.append(image_path)
            test_labels.append(class_id)
    
    # Predict and collect results
    y_true = []
    y_pred = []
    for image_path, label in zip(test_images, test_labels):
        predict = model(image_path)
        names_dict = predict[0].names
        probs = predict[0].probs.data.tolist()
        predicted_class = np.argmax(probs)
        
        y_true.append(label)
        y_pred.append(predicted_class)
    
    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=classes)
    print(report)

# Define classes
classes = ['Apitong', 'Balete', 'Bayabas', 'Guyabano', 'Kamagong', 'Langka', 'Mahogany', 'Mangga', 'Palo Maria']

# Generate classification report
generate_classification_report('runs/classify/train33', 'E:/Snapfolia - CS/YOLO v8/datasets/SPLITTED_DATA_YOLO_9_Samples', classes)
